In [1]:
import sys, os, glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
sys.path.append('../../..')
from src import data_path, direcslist, load_image, ImageQuant2
%matplotlib widget

### Data paths

In [ ]:
basepath = data_path + 'Imaging/mlc-4'
paths = direcslist(basepath, 1)

### Extract data from paths

In [ ]:
dates = []
lines = []
embryo_ids = []
for p in paths:
    code = p.strip('/').split('/')[-2]
    dates.append(code.split('_')[0])
    lines.append(code.split('_')[1])
    embryo_ids.append(int(os.path.getmtime(glob.glob(p + '/*488 SP 535-50*')[0])))
df = pd.DataFrame({'Path': paths, 'EmbryoID': embryo_ids, 'Line': lines, 'Date': dates})
df = df[df.Line.isin(['lp637', 'nwg240', 'nwg338', 'nwg376'])]
df.to_csv('df.csv')

### Segment

In [ ]:
# results = []
# for index, row in df.iterrows():
#     print(row.Path)
    
#     # Import movie
#     img = load_image(row.Path + '/af_corrected.tif')
#     roi = np.loadtxt(row.Path + '/ROI_fit_anaphase.txt')
    
#     # Run quantification
#     iq = ImageQuant2(img=img, roi=roi)
#     iq.segment()
    
#     # Save ROIs
#     for i, r in enumerate(iq.roi):
#         np.savetxt(row.Path + '/ROI_fit_%s.txt' % i, r)

### Import quantification model

In [ ]:
membg = np.loadtxt('../Quantification model calibration/membg6.txt')
cytbg = np.loadtxt('../Quantification model calibration/cytbg.txt')
calibration_factor = np.loadtxt('../Quantification model calibration/optogenetics_factors.txt')[6]
print(calibration_factor)

### Loop through embryos and quantify

In [ ]:
results = []
for index, row in df.iterrows():
    print(row.Path)
    
    # Import movie
    img = load_image(row.Path + '/af_corrected.tif')
    roi = [np.loadtxt(row.Path + '/ROI_fit_%s.txt' % i) for i in range(img.shape[0])]
    
    # Run quantification
    iq = ImageQuant2(img=img, roi=roi, cytbg=cytbg, membg=membg)
    iq.quantify(descent_steps=1000)
    iq.plot_losses(log=True)
    
    # Compile results
    res = iq.compile_res(ids=row.EmbryoID * np.ones(iq.nimgs), extra_columns={'Frame': np.arange(iq.nimgs)})
    res['Membrane signal'] *= calibration_factor
    res = res.astype({'Frame': int})
    results.append(res)

### Combine dataframes

In [ ]:
results_full = pd.concat(results)
results_full.to_csv('res.csv')

### Filter out polar body

In [ ]:
def nan_helper(y):
    return np.isnan(y), lambda z: z.nonzero()[0]

fig, ax = plt.subplots()
results_full = pd.read_csv('res.csv')
res_grouped = results_full.groupby(['EmbryoID', 'Frame'])
for i, (g, r) in enumerate(res_grouped):
    path = df[df.EmbryoID == g[0]]['Path'].values[0]
    roi = np.loadtxt(path + '/ROI_fit_%s.txt' % g[1])
    _mems = r['Membrane signal']
    mems = _mems.to_numpy()
    if os.path.exists(path + '/polar_body.txt'):
        polar_body = np.loadtxt(path + '/polar_body.txt')
        ind = int(mems.shape[0] * np.argmin(np.linalg.norm(roi - polar_body, axis=1)) / roi.shape[0])
        mems[ind-20:ind+20] = np.nan
        nans, x = nan_helper(mems)
        mems[nans] = np.interp(x(nans), x(~nans), mems[~nans])  
    ax.plot(mems)
    results_full.loc[_mems.index, 'Membrane signal'] = mems
results_full.to_csv('res_no_pb.csv')